In [18]:
from dotenv import load_dotenv
import os

load_dotenv()

OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]


In [19]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="mixtral-8x7b-instruct-fp16",
        max_tokens=200,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )

/Users/sangwonan/hackerthon/.venv/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [20]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.vectorstores import Milvus

In [21]:
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

In [22]:
vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": "localhost", "port": 19530},
    collection_name="city_data"
)

In [23]:
retriever = vector_store.as_retriever()

In [24]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [25]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [27]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [28]:
opt = widgets.ToggleButtons(
            options=['database', 'upload']
        )
question = widgets.Text(placeholder='Your question here')

In [29]:
question

Text(value='', placeholder='Your question here')

In [30]:
button_send = widgets.Button(
                description='Ask',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        # clear_output()
        print("You asked: ")
        print(f"{question.value}")

        print(f"Answer: ")
        print(chain.invoke(question.value))

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])
vbox_result